In [1]:
cd src

/Users/dynerman/work/special_ops_crispr_tools/dashit/src


In [2]:
import argparse
import flash
from Bio import SeqIO
from ortools.linear_solver import pywraplp
from common import Gene, Target

In [3]:
filename = "../../../dashit_data/human/RN45s.fasta"

In [18]:
    sequences = SeqIO.parse(open(filename, 'r'), 'fasta')

    genes = []
    
    for sequence in sequences:
        new_gene = Gene('hi')
        new_gene.seq = sequence.seq
        new_gene.targets = []
           
        for i in flash.kmers_range(new_gene.seq, 23):
            if 'G' == new_gene.seq[i+21] == new_gene.seq[i+22]:
                new_gene.targets.append(Target(flash.forward_20mer_at(new_gene.seq, i, 'F'),
                                               flash.cut_location((i, 'F'))))
            if 'C' == sequence.seq[i] == sequence.seq[i+1]:
                new_gene.targets.append(Target(flash.forward_20mer_at(new_gene.seq, i, 'R'),
                                               flash.cut_location((i, 'R'))))

        genes.append(new_gene)


hi  is missing a fasta file.


In [5]:
    solver = pywraplp.Solver('SolveIntegerProblem',
                             pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    objective = solver.Objective()

    sites_to_targets = {}

    for gene in sequences:
        for target_seq, target_loc in gene.targets:
            if target_seq not in sites_to_targets:
                sites_to_targets[target_seq] = [(gene, (target_seq,
                                                        target_loc))]
            else:
                sites_to_targets[target_seq].append((gene, (target_seq,
                                                            target_loc)))

    site_variables = {}

    for i, target_seq in enumerate(sites_to_targets):
        site_variables[target_seq] = solver.IntVar(0, 1, 'x_{}'.format(i))
        objective.SetCoefficient(site_variables[target_seq], 1)

    objective.SetMinimization()    


In [6]:
sequences = genes

In [8]:
min_spacing = 50
max_spacing = 200

In [9]:
    coverage_constraints = set()
    spacing_constraints = set()

    for sequence in sequences:
        for i, (seq, cut_loc) in enumerate(sequence.targets):        
            nearby_targets = [ s for s, l in sequence.targets if l != cut_loc and
                               abs(l - cut_loc) < min_spacing ]
        
            if len(nearby_targets) > 0:
                spacing_constraints.add(tuple(s for s in
                                              nearby_targets + [seq]))

            nearby_targets = [ s for s, l in sequence.targets if l != cut_loc and
                               abs(l - cut_loc) < max_spacing ]
        
            if len(nearby_targets) > 0:
                coverage_constraints.add(tuple(s for s in
                                               nearby_targets + [seq])) 


KeyboardInterrupt: 

In [19]:
len(genes)

1

In [17]:
genes[3].seq

Seq('GCTGACACGCTGTCCTCTGGCGACCTGTCGCTGGAGAGGTTGGGCCTCCGGATG...GCG', SingleLetterAlphabet())

In [20]:
import pysam

In [22]:
?pysam.AlignedSegment

In [111]:
header = { 'HD': {'VN':'1.0'}, 'SQ': [{'LN': 2000, 'SN': 'hello'}]}

In [112]:
outf = pysam.AlignmentFile("test.bam", "wb", header=header)

In [113]:
a = pysam.AlignedSegment()
a.query_name = 'what what'
a.query_sequence = 'GCTGACACGCTGTCCTCTGGCGACCTGTCGCTGGAGAGGTTGGGCCTCCGGATGCGCGCG'
a.reference_start = 0
a.cigar = ((0, 60),)
a.template_length = 60
a.reference_id = 0
a.next_reference_id = 0
a.next_reference_start=199
a.mapping_quality = 20

a.query_qualities = pysam.qualitystring_to_array("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<:<9/,&,22;;<<<")
a.tags = (("NM", 1),("RG", "L1"))

In [114]:
outf.write(a)

150

In [115]:
outf.close()

In [106]:
?pysam.AlignedSegment.cigartuples

In [46]:
len('AGCTTAGCTAGCTACCTATATCTTGGTCTTGGCCG')

35